# Boosting

## Outline

- Model Averaging
    - Bagging
    - Boosting
- Stagewise Additive Modeling
- Boosting and Logistic Regression
- MART
- Example: Predicting e-mail spam

Methods for improving the performance of weak learners. Classification
trees are adaptive and robust, but do not generalize well. The techniques
discussed here enhance their performance considerably.

## Classification Problem

Data $(X,Y) \in R^p \times \{0,1\}$

X is predictor, feature; $Y$ is class label, response.

$(X,Y)$ have joint probability distribution $D$.

Goal: Based on $N$ training pairs $(X_i,Y_i)$ drawn from $D$ produce a 
classifier $\hat{C(X)} \in \{0,1\}$

Goal: Choose $\hat{C}$ to have low generalization error
$$
R(\hat{C}) = P_D(\hat{C}(X)\neq Y) = E_D[\textbf{1}_{\hat{C}(X)\neq Y}]
$$

## Deterministic Concepts


$X \in R^p$ has distribution $D$.

C(X) is deterministic function $\in$ concept class.

Goal: Based on $N$ training pairs $(X_i,Y_i=C(X_i))$ drawn from $D$ produce a 
classifier $\hat{C(X)} \in \{0,1\}$

Goal: Choose $\hat{C}$ to have low generalization error
$$
R(\hat{C}) = P_D(\hat{C}(X)\neq Y) = E_D[\textbf{1}_{\hat{C}(X)\neq Y}]

## Model Averaging

Classification trees can be simple, but often produce noisy (bushy) or weak (stunted)
classifiers.
- Bagging: Fit many large trees to bootstrap-resampled versions of the training data, and classify by majority vote.
- Boosting: Fit many large or small trees or reweighted versions of the training data. Classify by weighted majority vote.

## Bagging

Bagging or bootstrap aggragation averages a given procedure over many samples, to reduce
its variance -- a poor man's Bayes.

Suppose $G(\textbf{X},x)$ is a classifier, such as a tree, producing a predicted class label
at input point x. To bag $G$, we draw bootstrap samples $\textbf{X}^{*1},...,\textbf{X}^{*B}$
each of size $N$ with replacement from the training data. Then
$$
\hat{G}_{\text{bag}}(x) = \text{Majority Vote} \{G(\textbf{X}^{*b}, x)\}_{b=1}^B
$$

Bagging can dramatically reduce the variance of unstable procedures (like trees), leading to
improved prediction. However any simple structure in $G$ (e.g. a tree) is lost. Bagging averages many trees, and produces smoother decision boundaries.

## Boosting

![title](img/boosting.png)

## AdaBoost

1. Initialize the observation weights
$$
w_i = \frac{1}{N} \text{, } i=1,2,..., N
$$
2. For $m=1$ to $M$ repeat steps (a)-(d):
    - a) Fit a classifier $G_m(x)$ to the training data using weights $w_i$
    - b) Compute
        $$
        err_m = \frac{\sum_{i=1}^N w_i I(y_i \neq G_m(x_i))}{\sum_{i=1}^N w_i}
        $$

    - c) Compute $\alpha = \log\left(\frac{1-err_m}{err_m}\right)$
    - d) Update weights for $i=1,..,N$:
        $$
        w_i \leftarrow w_i\cdot e^{\alpha_m \cdot I(y_i \neq G_m(x_i))}
        $$
        and renormalize to $w_i$ to sum to 1.
3. Output $G(x)= sign\left[\sum_{m=1}^M \alpha_m G_m(x)\right]$

## PAC Learning Model

$X \sim D$: Instance Space

$C: X \rightarrow \{0,1\}$ Concept

$h: X \rightarrow \{0,1\}$ Hypothesis

$error(h) = P_D[C(X) \neq h(X)]$

**Definition**: Consider a concept class defined over a set $X$ of length
$N$. $L$ is a learner (algorithm) using hypothesis space $H$. A concept class is PAC learn-able
by $Q$ using hypothesis space if for all $C$ in concept space, all distributions $D$ over $X$
and all $\epsilon,\delta\in (0,\frac{1}{2})$, learner $L$ will, with $Pr \geq (1-\delta)$, output
an $h$ in hypothesis space s.t. $error_D(h) \leq \epsilon$ in time polynomial in 
$\frac{1}{\epsilon},\frac{1}{\delta}, N$ and length of concept size.

Such an $L$ is called a strong Learner.

## Boosting a Weak Learner

Weak learner $L$ produces an $h$ with error rate 
$\beta = (\frac{1}{2}-\epsilon) < \frac{1}{2}$ with $Pr \geq (1-\delta)$ for any
distribution $D$.

$L$ has access to continuous stream of training data and a class oracle.
1. $L$ learns $h_1$ on first $N$ training points.
2. $L$ randomly filters the next batch of training points, extracting $\frac{N}{2}$ points correctly classified by $h_1$, $\frac{N}{2}$ incorrectly classified, and produces $h_2$.
3. $L$ builds a third training set of $N$ points for which $h_1$ and $h_2$ disagree, and produces $h_3$.
4. $L$ outputs $h=\text{Majority Vote}(h_1,h_2,h_3)$

$\textbf{Theorem}$: The Strength of Weak Learnability
$$
error_D(h) \leq 3\beta^2 - 2\beta^3 < \beta
$$

A stump is a two-node tree, after a single split. Boosting stumps works remarkably well on the nested-spheres problem.

Boosting drives the training error to zero. Further iterations continue to improve test error in many examples.

## Stagewise Additive Modeling

Boosting builds an additive model
$$
f(x) = \sum_{m=1}^M \beta_m b(x;\gamma_m)
$$
Here $b(x, \gamma_m)$ is a tree, and $\gamma_m$ parametrizes the splits.
We do things like that in statistics all the time!

With boosting, the parameters $(\beta_m, \gamma_m)$ are fit in a stagewise
fashion. This slows the process down, and tends to overfit less quickly.

## Stagewise Least Squares

Suppose we have available a basis family $b(x; \gamma)$ parametrized by $\gamma$.
For example, a simple family is $b(x; \gamma_j) = x_j$.
- After $m-1$ steps, suppose we have the model
$$
f_{m-1}(x) = \sum_{j=1}^{m-1} \beta_j b(x; \gamma_j)
$$
- At the $m$ step we solve
$$
\min_{\beta, \gamma} \sum_{i=1}^N (y_i - f_{m-1}(x_i)-\beta b(x_i;\gamma))^2
$$
- Denoting the residuals at the $m$th stage by $r_{im} = y_i - f_{m-1}(x_i)$, the previous step amount to 
$$
\min_{\beta, \gamma}(r_{im}-\beta b(x_i;\gamma))^2
$$
- Thus the term $\beta_m b(x; \gamma_m)$ the best fits the current residuals is added to the expansion at each step.

## Adaboost: Stagewise Modeling

- AdaBoost builds an additive logistic regression model
$$
f(x) = \log \frac{Pr(Y=1|x)}{Pr(Y=-1|x)} = \sum_{m=1}^M \alpha_m G_m(x)
$$ 
by stagewise fitting using the loss function
$$
L(y, f(x)) = e^{-yf(x)}
$$
- Given the current $f_{M-1}(x)$, our solution for $(\beta_m, G_m)$ is 
$$
arg\min_{\beta, G} \sum_{i=1}^N e^{-y_i(f_{m-1}(x_i)+\beta G(x))}
$$
where $G_m(x)\in\{-1, 1\}$ is a tree classifier and $\beta_m$ is a coefficient.
- With $w_i^{(m)} = e^{-y_i f_{m-1}(x_i)}$, this can be re-expressed as
$$
arg\min_{\beta, G} \sum_{i=1}^N w_i^{(m)} e^{-\beta y_iG(x_i)}
$$
- We can show that this leads to the Adaboost algorithm

## Why Exponential Loss?

- $e^{-yF(x)}$ is a monotone, smooth upper bound on misclassification loss at x.
- Leads to simple reweighting scheme.
- Has logit transform as population minimizer
$$
f^*(x) = \frac{1}{2} \log \frac{Pr(Y=1|x)}{Pr(Y=-1|x)}
$$
- Other more robust loss functions, like binomial deviance

## General Stagewise Algorithm

We can do the same for more general loss functions, not only least squares.
1. Initialize $f_0(x) = 0$
2. For $m=1$ to $M$:
    - Compute
    $$
    (\beta_m, \gamma_m) = arg\min_{\beta, \gamma} \sum_{i=1}^N L(y_i, f_{m-1}(x_i)+\beta b(x_i; \gamma))
    $$
    - Set $f_m(x) = f_{m-1}(x) + \beta_m b(x;\gamma_m)$ Sometimes we replace the previous step by 
    - Set $f_m(x) = f_{m-1}(x) + \nu \beta_m b(x;\gamma_m)$
    Here $\nu$ is a shrinkage factor, and often $\nu < 0.1$

## MART

- General boosting algorithm that works with a variety of different loss functions. Models include regression, outlier-resistant regression, K-class classification and risk modeling.
- MART uses gradient boosting to build additive tree models, for example, for representing the logits in logistic regression.
- Tree size is a parameter that determines the order of interaction.
- MART inherits all the good features of trees (variable selection, missing data, mixed predictors), and improves on the weak feature, such as prediction performance.

## MART in detail

Model
$$
f_M(x) = \sum_{i=1}^M T(x; \Theta_m)
$$
Where $T(x;\Theta)$ is a tree:
$$
T(x; \Theta) = \sum_{j=1}^J \gamma_j I(x\in R_j)
$$
with parameters $\Theta = \{ R)j, \gamma_j\}_1^J$

#### Fitting Criterion
Given $\hat{\Theta}_j$, $j=1,...,m-1$, we obtain $\hat{\Theta}_m$ via stagewise optimization:
$$
arg\min_{\Theta_m}\sum_{i=1}^N L(y_i, f_{m-1}(x_i)+T(x_i;\Theta_m))
$$
For general loss functions this is a very difficult optimization problem. Gradient boosting is an approximate 
gradient descent method.

## Gradient Boosting

1. Initialize $f_0(x) = arg\min_{\gamma} \sum_{i=1}^N L(y_i,\gamma)$.
2. For $m=1$ to $M$:
    - For $i=1,2,..., N$ compute
    $$
    r_{im} = - \left[ \frac{\partial L(y_i, f(x_i))}{\partial f(x_i)} \right]_{f=f_{m-1}}
    $$
    - Fit a regression tree to the targets $r_{im}$ giving terminal regions
    $$
    R_{jm}, j=1,2,...,J_m
    $$
    - For $j=1,2,...,J_m$ compute
    $$
    \lambda_{jm} = arg\min_{\gamma} \sum_{x_i\in R_{jm}} L(y_i, f_{m-1}(x_i)+\gamma)
    $$
    - Update
    $$
    f_m(x) = f_{m-1}(x) + \sum_{j=1}^{J_m}\gamma_{jm} I(x\in R_{jm})
    $$
3. Output $\hat{f}(x) = f_M(x)$

## Tree Size

The tree size $J$ determines the interaction order of the model:
$$
\eta(X) = \sum_j \eta_j(X_j) + \sum_{jk} \eta_{jk}(X_j, X_k) + \sum_{jkl} \eta_{jkl}(X_j, X_k, X_l) + \cdots 
$$

## Example: Predicting e-mail spam

- data from 4601 email messages
- goal: predict whether an email message is spam (junk email) or good.
- input features: relative frequencies in a message of 57 of the most commonly occuring words and punctuation marks in all the training the email messages.
- for this problem not all errors are equal; we want to avoid filtering out good email, while letting spam get through is not desirable but less serious in its consequences.
- we coded spam as 1 and email as 0
- A system like this would be trained for each user separately (e.g. their word lists would be different)

## Predictors

- 48 quantitative predictors-the percentage of words in the email that match a given word. Examples include `business`, `address`, `internet`, `free`, and `george`. The idea was that these could be customized for individual users.
- 6 quantitative predictors-the percentage of characters in the email that match a given character. The characters are `ch`; `ch(`, `ch[`, `ch!`, `ch$`, and `ch#`.
- The average length of uninterrupted sequences of capital letters: CAPAVE.
- The length of the longest uninterrupted sequence of capital letters: CAPMAX.
- The sum of the length of uninterrupted sequences of capital letters: CAPTOT.

## Some important features

39% of the training data were spam. 

Average percentage of words or characters in an email message equal to the indicated word or character.
We have chosen the words and characters showing the largest difference between spam and email.

## Spam Example Results

with 3000 training and 1500 test observations, MART fits an additive logistic model
$$
f(x) = \log \frac{Pr(spam|x)}{Pr(email|x)}
$$
using trees with $J=6$ terminal-node trees. MART achieves a test error of 4%, compared to 5.3% for an additive GAM, 5.5% for MARS,
and 8.7% for CART.